<a href="https://colab.research.google.com/github/Ramubala/image-classification/blob/main/Image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/Ramubala/image-classification.git

Cloning into 'image-classification'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.


In [ ]:
%ls
%cd image-classification

image-classification/  sample_data/
/content/image-classification


In [ ]:
import torchvision.datasets as datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import numpy as np

In [ ]:
dir = 'data'
#!rm -rf data
data = datasets.MNIST(root=dir, train=True, download =True, transform=ToTensor())
test_data = datasets.MNIST(root=dir, train=False, download =True, transform=ToTensor())

In [ ]:
type(data), len(data), len(test_data)

(torchvision.datasets.mnist.MNIST, 60000, 10000)

In [ ]:
train_data, valid_data = torch.utils.data.random_split(data, [50000, 10000])

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(valid_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64)

In [ ]:
class Simple_nn(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(in_features=784,out_features=250)
    self.layer2 = nn.Linear(in_features=250,out_features=100)
    self.layer3 = nn.Linear(in_features=100,out_features=10)
  
  def forward(self, input):
    layer1_output = torch.nn.functional.relu(self.layer1(input))
    layer2_output = torch.nn.functional.relu(self.layer2(layer1_output))
    layer3_output = torch.nn.functional.relu(self.layer3(layer2_output))
    return layer3_output,layer2_output

In [ ]:
device =  torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Simple_nn().to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
def get_accuracy(y_pred,y_actual):
  y_pred = torch.argmax(y_pred,1)
  accuracy = (y_pred == y_actual).sum()
  return 100*accuracy/y_pred.shape[0]

In [ ]:
def train_model(dataloader,model):
  train_loss=0
  train_accuracy = 0
  for input, y in iter(dataloader):
    optimizer.zero_grad()
    batch_size = input.shape[0]
    image_size = input.shape[2]*input.shape[3]
    input = input.squeeze(1).reshape(batch_size,image_size)
    output, h2 = model(input)
    loss = criterion(output, y)
    accuracy = get_accuracy(output,y)
    loss.backward()
    train_loss+=loss
    train_accuracy+=accuracy
    optimizer.step()
  train_loss=train_loss/len(dataloader)
  train_accuracy=train_accuracy/len(dataloader)
  return train_loss,train_accuracy

In [ ]:
def eval_model(dataloader,model):
    val_loss=0
    val_accuracy = 0  
    with torch.no_grad():
        for input, y in iter(dataloader):
          batch_size = input.shape[0]
          image_size = input.shape[2]*input.shape[3]
          input = input.squeeze(1).reshape(batch_size,image_size)
          output, h2 = model(input)
          loss = criterion(output, y)
          accuracy = get_accuracy(output, y)
          val_loss+=loss
          val_accuracy+=accuracy
        val_loss=val_loss/len(dataloader)
        val_accuracy=val_accuracy/len(dataloader)
        return val_loss,val_accuracy

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_epochs =20
import time
min_val_loss= float('inf')
for i in range(N_epochs):
  start_time = time.monotonic()
  train_loss, train_accuracy = train_model(train_dataloader, model)
  val_loss, val_accuracy = eval_model(val_dataloader, model)
  if val_loss <min_val_loss:
    min_val_loss = val_loss
    torch.save(model.state_dict(), 'tut1-model.pt')
  end_time = time.monotonic()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  print(f'Epoch: {i+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s | Train loss: {train_loss:02}, Train accuracy: {train_accuracy:02}, Val accuracy: {val_accuracy:02},  Val loss: {val_loss:02}')

Epoch: 01 | Epoch Time: 0m 9s | Train loss: 0.7662630081176758, Train accuracy: 73.22969818115234, Val accuracy: 76.5625,  Val loss: 0.6239291429519653
Epoch: 02 | Epoch Time: 0m 9s | Train loss: 0.5676634311676025, Train accuracy: 77.79331970214844, Val accuracy: 77.63734436035156,  Val loss: 0.5699832439422607
Epoch: 03 | Epoch Time: 0m 9s | Train loss: 0.5289758443832397, Train accuracy: 78.65249633789062, Val accuracy: 77.8463363647461,  Val loss: 0.5493590235710144
Epoch: 04 | Epoch Time: 0m 9s | Train loss: 0.5074461102485657, Train accuracy: 79.10205841064453, Val accuracy: 77.83638763427734,  Val loss: 0.5436131358146667
Epoch: 05 | Epoch Time: 0m 9s | Train loss: 0.4957595467567444, Train accuracy: 79.34382629394531, Val accuracy: 78.21456909179688,  Val loss: 0.5385439991950989
Epoch: 06 | Epoch Time: 0m 9s | Train loss: 0.48728689551353455, Train accuracy: 79.56961059570312, Val accuracy: 78.44347381591797,  Val loss: 0.5437370538711548
Epoch: 07 | Epoch Time: 0m 10s | Train